<a href="https://colab.research.google.com/github/julianox5/Tarefas-de-Programacao-Curso-Machine-Learning-do-GoogleDevelopers/blob/master/Regressao_Linea_com_Dados_Reais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##

##Regressão linear com um conjunto de dados real

Este Colab usa um conjunto de dados real para prever os preços das casas na Califórnia.

##Objetivos de aprendizado
Ao final desse colab, você aprenderá fazer o seguinte: 
* Ler um arquivo .csv em um DataFrame do Pandas
* Examinar um conjunto de dados
* experimentar diferentes recursos na contrução de um modelo 
* Ajustar hiperparâmetros do modelo


## Conjunto de dados Utilizado
O conjunto de dados para este exercício é baseado nos dados do censo de 1990 da Califórnia. O conjunto de dados é antigo, mas ainda oferece uma ótima oportunidade para aprender sobre programação de aprendizado de máquina.


# Usar a versão correta do TensorFlow
A seguinte célula de código oculto garante que o Colab seja executado no TensorFlow 2.X.

In [0]:
%tensorflow_version 2.x 

##Importar modulos relevantes para a programação nesse colab

In [0]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt 
# Granularidade é a extensão à qual um sistema é dividido em partes 
# pequenas, ou o sistema propriamente dito ou sua descrição ou observação.
# As linhas a seguir ajustam a granularidade da notificação

pd.options.display.max_rows = 10
pd.options.display.float_format = "{: .1f}".format

## Conjunto de dados .csv
Os conjuntos de dados geralmente são armazenados em disco ou em uma texto alternativoURL no formato .csv .

Um arquivo .csv bem formado contém nomes de colunas na primeira linha, seguidos por muitas linhas de dados. Uma vírgula divide cada valor em cada linha. Por exemplo, aqui estão as cinco primeiras linhas do arquivo .csv que contém o Conjunto de Dados de Habitação da Califórnia:
```
"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"
-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000
-114.470000,34.400000,19.000000,7650.000000,1901.000000,1129.000000,463.000000,1.820000,80100.000000
-114.560000,33.690000,17.000000,720.000000,174.000000,333.000000,117.000000,1.650900,85700.000000
-114.570000,33.640000,14.000000,1501.000000,337.000000,515.000000,226.000000,3.191700,73400.000000
```

## Carregando um arquivo .csv em uma DataFrame do Pandas
Este Colab, como muitos programas de aprendizado de máquina, reúne o arquivo .csv e armazena os dados na memória como um Dataframe do pandas.

A seguinte célula de código importa o arquivo .csv para um DataFrame do pandas e dimensiona os valores no rótulo ( median_house_value):

In [49]:
# importando o conjunto de dados
meu_dataframe = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

#dimensionar nosso rótulo
meu_dataframe["median_house_value"] /= 1000.0

#Imprimindo as primeiras linhas do meu DataFrame
meu_dataframe.head()



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.3,34.2,15.0,5612.0,1283.0,1015.0,472.0,1.5,66.9
1,-114.5,34.4,19.0,7650.0,1901.0,1129.0,463.0,1.8,80.1
2,-114.6,33.7,17.0,720.0,174.0,333.0,117.0,1.7,85.7
3,-114.6,33.6,14.0,1501.0,337.0,515.0,226.0,3.2,73.4
4,-114.6,33.6,20.0,1454.0,326.0,624.0,262.0,1.9,65.5


## Examinaminar o conjunto de dados
Uma grande parte da maioria dos projetos de aprendizado de máquina é conhecer seus dados. A API do pandas fornece uma função `describe()` que gera as seguintes estatísticas sobre cada coluna no DataFrame:`
* `count`, que é o número de linhas nessa coluna
*  `mean` e `std`, que contêm a média e o desvio padrão dos valores em cada coluna
* `min` e `max`, que contêm os valores mais alto e mais baixo em cada coluna
* `25%`, `50%`, `75%` Que contêm vários quantis .

In [50]:
# obtendo estatísticas úteis sobre meu conjuto de dados 
meu_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


## Anomalias encontradas no conjuto de dados
 O valor máximo (máximo) de várias colunas parece muito
 alto comparado aos outros quantis. Por exemplo
 a coluna total_rooms. Dado o quantil
 valores (25%, 50% e 75%), você pode esperar o
 valor máximo de total_ quartos a ser aproximadamente
5.000 ou possivelmente 10.000. No entanto, o valor máximo
 é realmente 37.937.

 Quando você vê anomalias em uma coluna, tenha mais cuidado
 sobre o uso dessa coluna como um recurso. Dito isto,
 anomalias em recursos potenciais às vezes espelham
 anomalias no rótulo, o que poderia tornar a coluna
 seja (ou pareça ser) um recurso poderoso.
 Além disso, como você verá mais adiante, você
 pode representar (pré-processar) dados brutos
 para transformar colunas em recursos úteis.

#Definir funções que constroem e treinam um modelo

O código a seguir define duas funções:

* `contruindo_modelo(taxa_aprendizado)`, que cria um modelo inicializado aleatoriamente.
* `treinando_modelo(modelo, recuros, rotulo, epochs)`, que treina o modelo a partir dos exemplos (recurso e rotulos) que você passa.

In [40]:
def contruindo_modelo(taxa_aprendizado):
  # A maioria dos modelos simples do tf.keras é sequencial
  modelo = tf.keras.models.Sequential()
  
  #descrevendo a topografia do modelo
  #seria apenas um nó em uma única camada
  modelo.add(tf.keras.layers.Dense(units=1,
                                   input_shape=(1,)))
  
  #compilando a topografia do modelo que o TensorFlow consiga executar
  modelo.compile(optimizer=tf.keras.optimizers.RMSprop(lr=taxa_aprendizado),
                 loss="mean_squared_error",
                 metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return modelo

def treinando_modelo(modelo, meu_df, recurso, rotulo, epochs, tam_lote):
  #alimentar o modelo como recursos e rotulos
  #o modelo treinará para o número especificado de épocas
  history = modelo.fit(x=meu_df[recurso],
                       y=meu_df[rotulo],
                       batch_size = tam_lote,
                       epochs = epochs)
  
  #Reunindo o peso e o viés do model 
  peso_treinado = modelo.get_weights()[0]
  vies_treinado = modelo.get_weights()[1]  

  #lista de épocas será separada do resto da historia
  epochs = history.epoch

  #isolando o erro para cada época
  hist = pd.DataFrame(history.history)

  # Acompanhar o treinamento imprimindo o erro médio quadrático da raiz
  rmse = hist["root_mean_squared_error"]

  return peso_treinado, vies_treinado, epochs, rmse

print("Tudo Ok!")

Tudo Ok!


##Criar funções de plotagem
As seguintes funções matplotlib criam os seguintes gráficos:
* De dispersão do recurso vs rótulo e uma linha mostrando a saída do modelo treinado
* Curva de perda

In [53]:
def plot_modelo(peso_treinado, vies_treinado, recurso, rotulo):
  #Rotule os eixos
  plt.xlabel(recurso)
  plt.ylabel(rotulo)

  #Criando um gráfico de dispersão a partir de 200 pontos aleatórios do conjunto
  exemplos_aleatorios = meu_dataframe.sample(n=200)
  
  plt.scatter(exemplos_aleatorios[recurso], exemplos_aleatorios[rotulo])

  #Criando uma linha vermelha representando o modelo
  #Começando nas cordenadas (x=0, y=0) até (x=1, y=1)
  x0 = 0,
  y0 = vies_treinado

  x1= 500
  y1= vies_treinado + (peso_treinado * x1)

  plt.plot([x0, x1],[y0, y1], c='r')

  #Renderizando o gráfico de disperção e a linha vermelha
  plt.show

""" Criar uma curva de perda VX épocas"""
def plot_curva_perda(epochs, rmse):
  plt.figure()
  plt.xlabel("Épocas")
  plt.ylabel("Erro Médio Quadrático da Raiz")
  
  plt.plot(epochs, rmse, label="Perda")
  plt.legend() 
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()

print("Função ok")

Função ok


## Chamando as funções do modelo
Uma parte importante do aprendizado de máquina é determinar quais recursos se correlacionam com o rótulo . Por exemplo, os modelos de previsão de valor residencial da vida real geralmente contam com centenas de recursos e recursos sintéticos. No entanto, este modelo depende apenas de um recurso. Por enquanto, vou usar arbitrariamente total_roomscomo como recurso.

In [0]:
# Criando os hiperparâmetros.
taxa_aprendizado= 0.001
epochs = 30
tam_lote = 30 

#Especificar os recurso e rótulos 
meus_recurso = "total_rooms" #número total de quartos em uma cidade específica
meus_rotulo = "median_house_value" #Valor mediano de uma casa espécifica por bloco
#ou seja, criamos um modelo que prevê o valor da casa com base apenas no total de quartos

#Descartar qualquer versão pré-existente do modelo
meu_modelo = None

#invocando as funções
meu_modelo = contruindo_modelo(taxa_aprendizado)
peso, vies, epochs, rmse = treinando_modelo(meu_modelo, meu_dataframe,
                                            meus_recurso, meus_rotulo,
                                            epochs, tam_lote)

print("\n Peso aprendido para o modelo é %.4f" % peso)
print("\n Viés aprendido para o modelo é %.4f" % vies)

plot_modelo(peso, vies, meus_recurso, meus_rotulo)
plot_curva_perda(epochs, rmse)


##Use o modelo para fazer previsões
Você pode usar o modelo treinado para fazer previsões. Na prática, você deve fazer previsões sobre exemplos que não são usados ​​no treinamento . No entanto, para este exercício, você apenas trabalhará com um subconjunto do mesmo conjunto de dados de treinamento. Um exercício posterior da Colab explorará maneiras de fazer previsões sobre exemplos não utilizados no treinamento.
Definir a função de previsão para valor da casa:

In [0]:
def prever_valor_casa(n, recurso, rotulo):
  lote = meu_dataframe[recurso][10000:10000 + n]
  valores_previsto = meu_modelo.predict_on_batch(x=lote)

  print("Rótulo de recurso previsto")
  print("valor valor valor")
  print("Em milhares de dólares em milhares de dólares")
  print("---------------------------------------------")
  for i in range(n):
    print("%5.0f %6.0f %15.0f" % (meu_dataframe[recurso][i],
                                  meu_dataframe[rotulo],
                                  valores_previsto[i][0] ))
    


Agora chamar a função de previsão do valor da casa em 10 exemplos

In [0]:
prever_valor_casa(10, meus_recurso, meus_rotulo)

##Tarefa 1: Experimente um recurso diferente
O recurso ´total_rooms´ tinha apenas um pouco de poder preditivo. Um recurso diferente teria maior poder preditivo? Tente usar ´population´ como recurso em vez de ´total_rooms´.

Nota: Ao alterar os recursos, também pode ser necessário alterar os hiperparâmetros.

In [0]:
meus_recurso = "population"
taxa_aprendizado = 0.05
epochs =18
tam_lote = 3
meu_modelo = contruindo_modelo(taxa_aprendizado)

peso, vies, epochs, rmse = treinando_modelo(meu_modelo, meu_dataframe,
                                            meus_recurso, meus_rotulo,
                                            epochs, tam_lote)

plot_modelo(peso, vies, meus_recurso, meus_rotulo)
plot_curva_perda(epochs, rmse)

prever_valor_casa(10, meus_recurso, meus_rotulo)


#Tarefa 2: Criar um recurso sintético
Vimos que total_roomse e population não era um recurso útil. Ou seja, nem o número total de quartos em um bairro nem a população do bairro previram com sucesso o preço médio da habitação desse bairro. Talvez a proporção de total_roomspara populationpossa ter algum poder preditivo. Ou seja, talvez a densidade de blocos esteja relacionada ao valor médio da casa.

Para explorar essa hipótese, faça o seguinte:
1. Criar um recurso sintético
2. Ajuste os três hiperparâmetros.
3. Determine se esse recurso sintético produz um valor de perda menor do que qualquer um dos recursos individuais que você tentou anteriormente neste exercício.

In [0]:
meu_dataframe = ["rooms_per_person"] = meu_dataframe["total_rooms"] / meu_dataframe["population"]
meus_recurso = "rooms_per_person"

taxa_aprendizado = 0.06
epochs = 24
tam_lote = 30

meu_modelo = contruindo_modelo(taxa_aprendizado)

peso, vies, epochs, rmse = treinando_modelo(meu_modelo, meu_dataframe,
                                            meus_recurso, meus_rotulo,
                                            epochs, tam_lote)

plot_modelo(peso, vies, meus_recurso, meus_rotulo)
plot_curva_perda(epochs, rmse)

prever_valor_casa(10, meus_recurso, meus_rotulo)

Com base nos valores de perda, esse recurso sintético produz um modelo melhor do que os recursos individuais que você tentou. No entanto, o modelo ainda não está criando ótimas previsões.

##Tarefa 3: Encontre os recursos cujos valores brutos se correlacionam com o rótulo
Até agora, contamos com tentativa e erro para identificar possíveis recursos para o modelo. Vamos confiar nas estatísticas.

Uma matriz de correlação indica como os valores brutos de cada atributo se relacionam com os valores brutos dos outros atributos. Os valores de correlação têm os seguintes significados:

* `1.0`: correlação positiva perfeita; isto é, quando um atributo aumenta, o outro atributo aumenta.
* `-1.0`: correlação negativa perfeita; isto é, quando um atributo aumenta, o outro atributo cai.
* `0.0`:  nenhuma correlação; as duas colunas não são linearmente relacionadas .
Em geral, quanto maior o valor absoluto de um valor de correlação, maior seu poder preditivo. Por exemplo, um valor de correlação de -0,8 implica muito mais poder preditivo do que uma correlação de -0,2.

A célula de código a seguir gera a matriz de correlação para atributos do Conjunto de Dados de Habitação da Califórnia:

In [0]:
meu_dataframe.corr()

In [0]:

meus_recurso = "median_income"

taxa_aprendizado = 0.06
epochs = 24
tam_lote = 30

meu_modelo = contruindo_modelo(taxa_aprendizado)

peso, vies, epochs, rmse = treinando_modelo(meu_modelo, meu_dataframe,
                                            meus_recurso, meus_rotulo,
                                            epochs, tam_lote)

plot_modelo(peso, vies, meus_recurso, meus_rotulo)
plot_curva_perda(epochs, rmse)

prever_valor_casa(10, meus_recurso, meus_rotulo)


O recurso `median_income` correlaciona 0.7 com o rótulo
(median_house_value), então median_income` pode ser um
bom recurso. As outras sete características possíveis
todos têm uma correlação relativamente próxima de 0.

